In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from math import log2
%matplotlib inline

from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_poisson_deviance
from sklearn.metrics import mean_gamma_deviance
from sklearn.metrics import median_absolute_error
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.pipeline import make_pipeline
import time
import matplotlib as mpl

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor



from sklearn.preprocessing import StandardScaler,PowerTransformer,MinMaxScaler,QuantileTransformer,normalize
from sklearn.model_selection import train_test_split, learning_curve, ShuffleSplit
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

import pathlib
import os
import sys

# for selection the right path
import os,sys,inspect,pathlib
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from common.model_trainer_reg import *
from common.regression_plotfunctions import *

from GD.LinearRegression import LinearRegression
from KNN.KNNRegressor import KNNRegressor

import supercon_preprocessing

In [2]:
# Custom functions

module_path = pathlib.Path(os.getcwd()).parent
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))
print(sys.path)

from common.dataset_grabber import get_data_path

#https://joeganser.github.io/2019-01-26-superconductivity-regression/

['/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2', '/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2/Superconductivity', '', '/home/alexander/ngsuite/ngsolve-install/lib/python3/dist-packages', '/home/alexander/ngsuite/ngsolve-install/bin', '/home/alexander/.local/bin', '/usr/local/sbin', '/usr/local/bin', '/usr/sbin', '/usr/bin', '/sbin', '/bin', '/usr/games', '/usr/local/games', '/snap/bin', '/home/alexander/.vscode/extensions/ms-toolsai.jupyter-2020.11.399280825/pythonFiles', '/home/alexander/.vscode/extensions/ms-toolsai.jupyter-2020.11.399280825/pythonFiles/lib/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/home/alexander/.local/lib/python3.7/site-packages', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-

In [3]:
datapath = get_data_path("Superconduct", "train.csv")
display(datapath)

PosixPath('/home/alexander/Dokumente/WS2020/MaschinLearning/ML_2020/Ex2/Datasets/Superconduct/train.csv')

In [4]:
df_raw = pd.read_csv(datapath)

df_raw = df_raw.sample(3000)

# Train and Test

In [5]:
Y, X = supercon_preprocessing.preprocessing(df_raw, transform=True)

## SGD-Regression

In [6]:
MODEL = SGDRegressor
NAME = "SGD"
PATH = "out/"+NAME+"/"
params = {"alpha" : [0.0001]}
n_splits = 2
test_size = 0.3

modeltrainer = ModelTrainer(MODEL, params, X, Y, thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = test_size)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_SGD.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = test_size, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_SGD.csv")
display(results)

,alpha,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train
0,0.0001,0.014775,0.000383,0.658317,0.038196,24,3000,900,2100


,alpha,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train
0,0.0001,0.026304,0.001213,1,0.608339,0.043782,24,3000,900,2100
1,0.0001,0.024738,0.000463,2,0.661256,0.039820,24,3000,900,2100


## My SGD-Regression

In [7]:
MODEL = LinearRegression
params = {"alpha" : [0.0001], "max_iter": [1000]}

modeltrainer = ModelTrainer(MODEL, params, X, Y, thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = test_size)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "my_TTS_SGD.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = test_size, random_state = 42)
results = modeltrainer.retResults(PATH + "my_CV_SGD.csv")
display(results)

,alpha,max_iter,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train
0,0.0001,1000,0.560466,0.000176,0.671918,0.036675,24,3000,900,2100


,alpha,max_iter,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train
0,0.0001,1000,0.810737,0.000447,1,0.671918,0.036675,24,3000,900,2100
1,0.0001,1000,0.507704,0.000220,2,0.692176,0.036185,24,3000,900,2100


## KNN-Regression

In [8]:
MODEL = KNeighborsRegressor
params = {"weights" : ["uniform"]}
NAME = "KNN"
PATH = "out/"+NAME+"/"

modeltrainer = ModelTrainer(MODEL, params, X, Y, thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = test_size)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_KNN.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = test_size, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_KNN.csv")
display(results)

,weights,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train
0,uniform,0.02316,0.223479,0.797647,0.02262,24,3000,900,2100


,weights,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train
0,uniform,0.019508,0.200181,1,0.797647,0.022620,24,3000,900,2100
1,uniform,0.016992,0.218872,2,0.818515,0.021334,24,3000,900,2100


In [9]:
MODEL = KNNRegressor
params = {"n_neighbors" : [5],
           "p": [2]}
NAME = "KNN"
PATH = "out/"+NAME+"/"

modeltrainer = ModelTrainer(MODEL, params, X, Y, thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = test_size)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "my_TTS_KNN.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = test_size, random_state = 42)
results = modeltrainer.retResults(PATH + "my_CV_KNN.csv")
display(results)

chunking...


,n_neighbors,p,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train
0,5,2,0.000012,1.083348,0.79653,0.022745,24,3000,900,2100


chunking...
chunking...


,n_neighbors,p,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train
0,5,2,0.000007,1.077107,1,0.796530,0.022745,24,3000,900,2100
1,5,2,0.000006,1.083376,2,0.819079,0.021268,24,3000,900,2100


## RF-Regression

In [10]:
MODEL = RandomForestRegressor
params = {"n_estimators" : [100]}
NAME = "RF"
PATH = "out/"+NAME+"/"

modeltrainer = ModelTrainer(MODEL, params, X, Y, thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = test_size)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_DT.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = test_size, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_DT.csv")
display(results)

,n_estimators,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train
0,100,10.210868,0.028167,0.856087,0.016088,24,3000,900,2100


,n_estimators,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train
0,100,10.254238,0.027609,1,0.858685,0.015797,24,3000,900,2100
1,100,10.071617,0.027184,2,0.867895,0.015529,24,3000,900,2100


## DT-Regression

In [11]:
MODEL = DecisionTreeRegressor
params = {"criterion": ["mse"]}
NAME = "DT"
PATH = "out/"+NAME+"/"


modeltrainer = ModelTrainer(MODEL, params, X, Y, thread_cnt=1)
########### train with TrainTestSplit  ###################
modeltrainer.TTSplit(test_size = test_size)
modeltrainer.train()
results = modeltrainer.retResults(PATH + "sklearn_TTS_DT.csv")
display(results)
############ shuffle_Cross validation  ###################
modeltrainer.CV_shuffle_split(k = n_splits, test_size = test_size, random_state = 42)
results = modeltrainer.retResults(PATH + "sklearn_CV_DT.csv")
display(results)

,criterion,train_time,inference_time,R2_score,RMSE,D,N,n_test,n_train
0,mse,0.178356,0.000559,0.694593,0.03414,24,3000,900,2100


,criterion,train_time,inference_time,k,R2_score,RMSE,D,N,n_test,n_train
0,mse,0.171164,0.000551,1,0.718882,0.031425,24,3000,900,2100
1,mse,0.171548,0.000679,2,0.758449,0.028395,24,3000,900,2100
